# Customer Churn Prediction DataSC project

## Introduction

In this project for DataSC, we focus on aiming to predict customer churn using machine learning techniques.

*Dataset Source: [Telco Customer Churn](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)*


## 1. Data Loading

Importing necessary libararies:

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()


Please work


## 2. Data Exploration

In [ ]:
df.info()